In [31]:
from ModelDataGenerator import ModelDataGenerator
from ModelBuilder import ModelBuilder
import os

In [32]:
os.makedirs('models/test_model/data', exist_ok=True)
os.makedirs('models/test_model/data', exist_ok=True)

In [33]:
# Create an instance of ModelDataGenerator
generator = ModelDataGenerator(
    image_size=30, 
    num_images=2000,
    dataset_path='models/test_model/data'
)

# Generate and save the dataset
image_paths, labels = generator.generate_dataset()

In [34]:
model = ModelBuilder(
    image_size=30, 
    num_classes=1, 
    image_paths=image_paths, 
    labels= labels, 
    dataset_path='models/test_model/data'
)
model.train_model(epochs=10)
model.evaluate_model()
model.save_model_as_h5(model_name='test_model')

Epoch 1/10
50/50 [==============================] - 2s 20ms/step - loss: 0.5864 - accuracy: 0.7312 - val_loss: 0.3117 - val_accuracy: 1.0000
Epoch 2/10
50/50 [==============================] - 1s 17ms/step - loss: 0.1016 - accuracy: 1.0000 - val_loss: 0.0153 - val_accuracy: 1.0000
Epoch 3/10
50/50 [==============================] - 1s 16ms/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.0039 - val_accuracy: 1.0000
Epoch 4/10
50/50 [==============================] - 1s 17ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 5/10
50/50 [==============================] - 1s 17ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 6/10
50/50 [==============================] - 1s 17ms/step - loss: 9.5019e-04 - accuracy: 1.0000 - val_loss: 7.6087e-04 - val_accuracy: 1.0000
Epoch 7/10
50/50 [==============================] - 1s 17ms/step - loss: 5.4153e-04 - accuracy: 1.0000 - val_loss: 1.8032e-04 - val_accuracy: 1.00

c:\Users\Fabio Bove\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [35]:
# Get the prediction from the model
data_for_predictions  = [
    "./models/test_model/pred_data/pred_test_is_1.png",
    "./models/test_model/pred_data/pred_test_is_0.png",
    "./models/test_model/pred_data/pred_test_is_0.png",
    "./models/test_model/pred_data/pred_test_is_0.png",
    "./models/test_model/pred_data/pred_test_is_1.png",
    "./models/test_model/pred_data/pred_test_is_1.png",
]

# Set a threshold for classification (you can adjust this based on your needs)
threshold = 0.5

for data in data_for_predictions:
    
    prediction = model.predict(data)
    # Check if the prediction is greater than the threshold
    if prediction[0][0] >= threshold:
        print("Model predicts a 1")
    else:
        print("Model does not predict a 1")
    correct_value = data.split('_')[-1].split('.')[0]
    print(f"values is a {correct_value}")


1/1 [==============================] - 0s 116ms/step
Model predicts a 1
values is a 1
1/1 [==============================] - 0s 23ms/step
Model does not predict a 1
values is a 0
1/1 [==============================] - 0s 23ms/step
Model does not predict a 1
values is a 0
1/1 [==============================] - 0s 56ms/step
Model does not predict a 1
values is a 0
1/1 [==============================] - 0s 56ms/step
Model predicts a 1
values is a 1
1/1 [==============================] - 0s 50ms/step
Model predicts a 1
values is a 1


In [36]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import time
import csv 


def calculate_size_in_bits(layer_output):
    # Get the data type of the layer output
    dtype = layer_output.dtype
    # Calculate the size in bits
    size_in_bits = tf.reduce_prod(layer_output.shape) * tf.constant(dtype.itemsize * 8)
    return size_in_bits.numpy()  # Convert to a Python scalar for writing to CSV

def create_analytics_csv(analytics_path, layer_name, layer_size, layer_inference_time):
    with open(analytics_path, mode='a', newline='') as csv_file:
        fieldnames = ['layer', 'layer_size', 'layer_inference_time']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Check if the file is empty and write the header if needed
        if csv_file.tell() == 0:
            writer.writeheader()

        writer.writerow({
            'layer': layer_name,
            'layer_size': layer_size,
            'layer_inference_time': layer_inference_time
        })
    print(f"\n{layer_name}, {layer_size}, {layer_inference_time}")

def load_model(model_path='models/test_model/test_model.h5'):
    return tf.keras.models.load_model(model_path)

def predict_single_layer(model, layer_id, layer_input_data, analytics_path):
    layer = model.layers[layer_id]
    # Create an intermediate model with the current layer
    intermediate_model = tf.keras.Model(inputs=layer.input, outputs=layer.output)
    print(layer.input, layer.output)
    # Predict using the current layer, keeps track of the time it takes
    t_begin = time.time() * 1000
    layer_output = intermediate_model.predict(layer_input_data)
    t_end = time.time() * 1000

    create_analytics_csv(
        analytics_path = analytics_path,
        layer_name = layer.name,
        layer_inference_time = t_end - t_begin,
        layer_size = calculate_size_in_bits(layer_output)
    )
    
    return layer_output

def perform_predict(model, start_layer_index=0, input_data='', analytics_path=''):
    num_layers = len(model.layers)
    predictions = []

    # Load and preprocess the input image for the first layer
    input_image = load_img(input_data, target_size=(30, 30))
    input_array = img_to_array(input_image)
    input_array = tf.expand_dims(input_array, 0)  # Create batch axis
    input_array = input_array / 255.0  # Normalize pixel values to be between 0 and 1

    for layer_id in range(start_layer_index, num_layers):
        # For the first layer, use the preprocessed input image
        if layer_id == 0:
            # Resize the input to match the expected shape of the first layer
            input_array = tf.image.resize(input_array, (30, 30))
            layer_output = predict_single_layer(model, layer_id, input_array, analytics_path)
        else:
            # For subsequent layers, reshape the output of the previous layer if needed
            previous_layer_output = predictions[-1]
            expected_input_shape = model.layers[layer_id].input_shape[1:]  # Exclude batch dimension
            if previous_layer_output.shape[1:] != expected_input_shape:
                previous_layer_output = tf.image.resize(previous_layer_output, expected_input_shape)
            layer_output = predict_single_layer(model, layer_id, previous_layer_output, analytics_path)

        predictions.append(layer_output)

    return predictions

model = load_model(model_path='./models/test_model/test_model.h5')
input_data = './models/test_model/pred_data/pred_test_is_0.png'
analytics_path = './models/test_model/analytics_data/analytics.csv'
predictions = perform_predict(model=model, input_data=input_data, analytics_path=analytics_path)
prediction = predictions[-1][0]
threshold= 0.5
# Check if the prediction is greater than the threshold
if prediction >= threshold:
    print("Model predicts a 1")
else:
    print("Model does not predict a 1")
correct_value = input_data.split('_')[-1].split('.')[0]
print(f"values is a {correct_value}")

KerasTensor(type_spec=TensorSpec(shape=(None, 30, 30, 3), dtype=tf.float32, name='layer_0_input'), name='layer_0_input', description="created by layer 'layer_0_input'") KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 32), dtype=tf.float32, name=None), name='layer_0/Relu:0', description="created by layer 'layer_0'")
1/1 [==============================] - 0s 57ms/step

layer_0, 802816, 98.31640625
KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 32), dtype=tf.float32, name=None), name='layer_0/Relu:0', description="created by layer 'layer_0'") KerasTensor(type_spec=TensorSpec(shape=(None, 14, 14, 32), dtype=tf.float32, name=None), name='layer_1/MaxPool:0', description="created by layer 'layer_1'")
1/1 [==============================] - 0s 91ms/step

layer_1, 200704, 176.215576171875
KerasTensor(type_spec=TensorSpec(shape=(None, 14, 14, 32), dtype=tf.float32, name=None), name='layer_1/MaxPool:0', description="created by layer 'layer_1'") KerasTensor(type_spec=TensorSpec(shape=(